**Финальный проект: E-commerce — Выявление профилей потребления.**

**Описание данных:** Датасет описывает транзакции интернет-магазина товаров для дома и быта «Пока все ещё тут».

**Задача:** Сегментировать покупателей по профилю потребления.

**1. ОТКРОЕМ ФАЙЛ И ИЗУЧИМ ИНФОРМАЦИЮ О ДАННЫХ.**

In [1]:
# импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as st
import numpy as np
import seaborn as sns; sns.set()
import plotly.express as px
from plotly.offline import iplot
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import silhouette_score
from pymystem3 import Mystem
from sklearn.cluster import KMeans

ModuleNotFoundError: No module named 'pymystem3'

In [ ]:
online_shop = pd.read_csv('https://code.s3.yandex.net/datasets/ecom_dataset_upd.csv')

In [ ]:
# выведем данные
online_shop.head(10)

In [ ]:
# выводим инфу
online_shop.info()

**2. ПРЕДОБРАБОТКА ДАННЫХ.**

In [ ]:
# посмотрим количество пропусков
online_shop.isna().sum()

In [ ]:
#провереяем дубликаты
online_shop.duplicated().sum()

In [ ]:
# проверим скрытые дубликаты
online_shop[['customer_id', 'order_id', 'product']].duplicated().sum() 

In [ ]:
# проверим скрытые дубликаты
#online_shop[['customer_id', 'product']].duplicated().sum()

In [ ]:
#удалим скрытые дубликаты 
online_shop = online_shop.drop_duplicates(subset = ['customer_id', 'order_id', 'product']).reset_index()

In [ ]:
# один заказ-один пользователь и в основном датасете оставляем только пользователей, которые в него не входят
online_shop_one = online_shop.groupby('order_id')['customer_id'].nunique().sort_values(ascending=False).reset_index()
online_shop_one.head(30)

In [ ]:
online_shop_one_l = online_shop_one.loc[online_shop_one['customer_id'] > 1]
online_shop_one_l = list(online_shop_one_l['order_id'])

In [ ]:
online_shop = online_shop[online_shop['order_id'].isin(online_shop_one_l)==False].reset_index(drop=True)

In [ ]:
online_shop.info()

- явных дубликатов нет;
- пропусков нет;
- типы данных соответствуют содержанию;
- всего 7474 строки.

In [ ]:
# переведем в нижний регистр названия товара 
online_shop['product'] = online_shop['product'].str.lower()

In [ ]:
# преобразуем столбец date и добавим столбец месяц
online_shop['date'] = pd.to_datetime(online_shop['date'], format='%Y%m%d%H%M')
online_shop['month'] = pd.DatetimeIndex(online_shop['date']).month

In [ ]:
#создадим столбец с вырочкой
online_shop['revenue'] = online_shop.quantity * online_shop.price

In [ ]:
# создадим столбец в днях
online_shop['day'] = online_shop['date'].dt.strftime('%Y-%m-%d')

In [ ]:
# проверяем
online_shop.head(10)

In [ ]:
# смотрим за какой периуд у нас данных
print('Рассматриваемый период: с', online_shop['date'].min().date(), "по" , online_shop['date'].max().date())

In [ ]:
online_shop.describe()

Среднее значение товаров - 2.5. Стоимость товара составляет от 9 у.е. до 14 917. Медиана - 150 у.е., среднее значение - 531.58. 
Имеется заказ с максимальным значением на 1000 единиц.

Посмотрим что это за товар на 1000 ед.

In [ ]:
online_shop[online_shop['quantity']==1000]

Покупатель с 1000 купленными вантузами это явная ошибка, удалим его (тестовый заказ, о котором говорили на встрече).

In [ ]:
# удалим эту аномалию
online_shop = online_shop.drop(online_shop[online_shop['customer_id'] == '312e9a3e-5fca-43ff-a6a1-892d2b2d5ba6'].index)

**Вывод:**

В таблице содержатся данные о заказах покупателей в интернет-магазине за период с 1 октября 2018 г. до 31 января 2020 г. 

Были добавлены столбцы с месяцем, днём и выручкой для дальнейшей работы.

Дубликатов и пропусков нет. Количество товаров варьируется от 1 до 1000. Среднее значение товаров - 2.5. Стоимость товара составляет от 9 у.е. до 14 917. Медиана - 150 у.е., среднее значение - 531.58. Имеется заказ с максимальным значением на 1000 единиц. 

Медиана и среднее значение у стоимости товаров и выручки различается в несколько раз, что свидетельствует о наличии выбросов. Был удален заказ на 1000 ед. товара. 

В среднем выручка составляет 68 760 у.е. за товарную позицию (не заказа). Самый крупный заказ составил 112 789  у.е.

**3. ИССЛЕДОВАТЕЛЬСКИЙ АНАЛИЗ ДАННЫХ.**

Посчитаем общие метрики: среднее количество заказов на одного покупателя, количество уникальных покупателей,количество уникальных товаров, количество уникальных заказов, средняя выручка с одного покупателя, средняя сумма заказа, среднее число позиций товаров в заказах, среднее число единиц товаров в заказах.

In [ ]:
online_shop['total_price'] = online_shop['quantity'] * online_shop['price']
online_shop['month'] = online_shop['date'].astype('datetime64[M]') 
median_order = online_shop.groupby('customer_id').agg({'order_id':'count'})

grouped_online_shop = online_shop.groupby(['customer_id', 'order_id'])\
.agg({'date':'first', 'product':'count', 'quantity':'sum','price':'mean', 'total_price':'sum'}).reset_index()\
.rename(columns={'date':'first_date_order', 'product':'cnt_unique_products_in_order', 
                 'quantity':'total_cnt_goods_in_order', 'price':'avg_product_price', 'total_price':'order_price'})

online_shop_customers = grouped_online_shop.groupby('customer_id')\
.agg({'order_id':'nunique', 'cnt_unique_products_in_order':'mean', 'total_cnt_goods_in_order':'mean', 
      'avg_product_price':'mean', 'order_price':['sum','mean']}).reset_index()

online_shop_customers.columns = ['customer_id', 'cnt_orders', 'avg_products_in_order', 'avg_goods_in_order', 
                                'avg_product_price', 'total_revenue', 'avg_order_price']

In [ ]:
# print('Cреднее количество заказов на одного покупателя: {:.0f} шт.'.format(median_order['order_id'].median()))
print('Общее количество уникальных покупателей: {:.0f}'.format(len(online_shop['customer_id'].unique())))
print('Общее количество уникальных товаров: {:.0f}'.format(len(online_shop['product'].unique())))
print('Количество уникальных заказов: {:.0f}'.format(len(online_shop['order_id'].unique())))

print('Медианный чек по всем покупателям: {:.0f} у.е.'.format((online_shop_customers['total_revenue']/online_shop_customers['cnt_orders']).median()))
print('Медианная выручка с одного покупателя: {:.0f} у.е.'.format(online_shop_customers['total_revenue'].median()))
print('Медианное число позиций товаров в заказах: {:.0f} шт.'.format(online_shop_customers['avg_products_in_order'].median()))
print('Медианное число единиц товаров в заказах: {:.0f} шт.'.format(online_shop_customers['avg_goods_in_order'].median()))

**Посмотрим количество единиц товаров в одной покупке**

In [ ]:
online_shop.groupby('order_id').agg({'quantity': 'sum'}).describe()

Заказы от 1 до 30 шт. товаров

In [ ]:
online_shop.groupby('order_id').agg({'quantity': 'sum'}).hist(range=(0, 30), bins=50)

plt.title('Количество единиц товаров в 1 покупке')
plt.locator_params(axis='x', nbins=20)
plt.xlabel('Количество товаров')
plt.ylabel('Количество покупок');

На графике видно, в одной покупке встречается чаще 1 единица товара.

**Изменение количества покупок в месяц**

In [ ]:
online_shop.groupby(['month']).agg({'order_id': 'nunique'})

In [ ]:
online_shop.groupby(['month']).agg({'order_id': 'nunique'}) \
    .plot(grid=True, legend=False)
plt.title('Изменение количества покупок в месяц')
plt.xlabel('Месяц')
plt.ylabel('Количество покупок');

Наибольшее количество заказов были сделаны покупателями в в декабре 2018 г., феврале 2019 г., в марте и апреле 2019 г. новогодние праздники и весна(начало дачного сезона). Большой провал в январе и июне. И снова подъем с ноября. 

**Выручка с одной покупки**

In [ ]:
online_shop.groupby('order_id').agg({'revenue': 'sum'}).describe()

In [ ]:
online_shop.groupby('order_id').agg({'revenue': 'sum'}).hist(range=(0, 7000), bins=50)

plt.title('Выручка с 1 покупки')
plt.locator_params(axis='x', nbins=10)
plt.xlabel('Выручка')
plt.ylabel('Количество покупателей');


95-й и 99-й перцентили выручки по заказам

In [ ]:
revenue_unique_orders = (
    online_shop.groupby('order_id', as_index=False)
    .agg({'revenue': 'sum'})
)
print(np.percentile(revenue_unique_orders['revenue'], [95, 99])) 


Средняя выручка или чек с заказа составляет 1121.37 у.е., медиана - 580 у.е. Минимальная выручка с покупки - 9 у.е., максимальная - 49 432 у.е.

5% заказов принесли выручку не менее 3768 у.е. 1% заказов - не менее 7506.1 у.е.

**Выручка на 1 покупателя**

In [ ]:
online_shop.groupby('customer_id').agg({'revenue': 'sum'}).describe()

In [ ]:
online_shop.groupby('customer_id').agg({'revenue': 'sum'}).hist(bins=100)

plt.title('Выручка на 1 покупателя')
plt.xlabel('Выручка')
plt.ylabel('Количество покупателей');


95-й и 99-й перцентили выручки по пользователям

In [ ]:
revenue_unique_customers = (
    online_shop.groupby('customer_id', as_index=False)
    .agg({'revenue': 'sum'})
)
print(np.percentile(revenue_unique_customers['revenue'], [95, 99])) 

Один покупатель принес за весь период в среднем 1623.02 у.е., медиана - 837 у.е. Минимальная сумма - 15 у.е., максимальная - 166988 у.е.

5% пользователей принесли выручку не менее 4885.7 у.е. 1% пользователей - не менее 9867.37 у.е.

**Изменение общей выручки по месяцам**

In [ ]:
online_shop.groupby(['month']).agg({'revenue': 'sum'})

In [ ]:
online_shop.groupby(['month']).agg({'revenue': 'sum'}) \
      .plot(grid=True)
plt.title('Изменение выручки по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Выручка');

Выручка в ноябре 2018 составила 359 348 у.е - самый высокий показатель. 

**Изменение среднего чека покупателя по месяцам**

In [ ]:
avg_check = online_shop.groupby(['month']).agg({'revenue':'sum','customer_id':'nunique'}).reset_index()
avg_check['avg_check'] = avg_check['revenue'] / avg_check['customer_id']
avg_check


In [ ]:
avg_check.plot(marker='o', x='month', y='avg_check', grid=True);

Самый большой средний чек покупателя в ноябре 2018 и составил 2018.80 у.е. хотя кол-во покупок не самое большое. 

**Вывод:** 
Наибольшее количество заказов были сделаны покупателями в в декабре 2018 г., феврале 2019 г., в марте и апреле 2019 г. новогодние праздники и весна(начало дачного сезона). Большой провал в январе и июне. И снова подъем с ноября.

Средняя выручка или чек с заказа составляет 1121.37 у.е., медиана - 580 у.е. Минимальная выручка с покупки - 9 у.е., максимальная - 49 432 у.е.
5% заказов принесли выручку не менее 3768 у.е. 1% заказов - не менее 7506.1 у.е.

Один покупатель принес за весь период в среднем 1623.02 у.е., медиана - 837 у.е. Минимальная сумма - 15 у.е., максимальная - 166988 у.е.
5% пользователей принесли выручку не менее 4885.7 у.е. 1% пользователей - не менее 9867.37 у.е.

Выручка в ноябре 2018 составила 359 348 у.е - самый высокий показатель Самый большой средний чек покупателя в ноябре 2018 и составил 2018.80 у.е. хотя кол-во покупок не самое большое.

**Рассчитаем продуктовые метрики DAU и MAU**

In [ ]:
dau_total = (online_shop.groupby(['day']).agg({'customer_id': 'nunique'}).mean())
print('Среднее количество уникальных покупателей в день:', int(dau_total))

mau_total = (online_shop.groupby(['month']).agg({'customer_id': 'nunique'}).mean())
print('Среднее количество уникальных покупателей в месяц:', int(mau_total))

sessions_per_user = online_shop.groupby('date').agg({'customer_id':'count'}).mean()
print('Среднее количество сессий в день:', int(sessions_per_user))

**Рассчитаем Retention Rate**

Определим время первого визита

In [ ]:
first_activity_date = online_shop.groupby('customer_id')['date'].min()
first_activity_date.name = 'first_activity_date'

In [ ]:
first_activity_date.to_frame().reset_index().head()


Первый визит за покупками был 2019-01-27 01:07.

Определим дату первой активности

In [ ]:
online_shop = online_shop.join(first_activity_date, on='customer_id', how='left')

In [ ]:
online_shop['first_activity_date'] = online_shop['first_activity_date'].astype('datetime64[M]')
online_shop['first_month'] = online_shop['date'].astype('datetime64[M]')

In [ ]:
online_shop['cohort_lifetime'] = online_shop['first_month'] - online_shop['first_activity_date']

In [ ]:
online_shop['cohort_lifetime'] = online_shop['cohort_lifetime'] / np.timedelta64(1,'M')

In [ ]:
online_shop['cohort_lifetime'] = online_shop['cohort_lifetime'].round().astype(int)

In [ ]:
# Сгруппируем данные когорты
cohorts = online_shop.groupby(['first_activity_date', 'cohort_lifetime']).agg({'customer_id':'nunique'}).reset_index()

In [ ]:
cohorts.head()

Первая активность была 01.10.2018

Чтобы найти Retention Rate, нужно сперва получить число пользователей, изначально бывших в когорте, и на него разделить число пользователей в каждую следующую неделю. Найдём исходное количество пользователей в когорте. Возьмём их число на нулевую неделю:

In [ ]:
initial_users_count = cohorts[cohorts['cohort_lifetime'] == 0][['first_activity_date', 'customer_id']]

In [ ]:
# Переименуем столбец customer_id
initial_users_count = initial_users_count.rename(columns={'customer_id':'cohort_users'}) 

In [ ]:
# Объединим данные по когортам с исходным количеством пользователей в когорте
cohorts = cohorts.merge(initial_users_count, on='first_activity_date')

In [ ]:
cohorts.head()

Рассчитаем Retention Rate. Разделим количество активных пользователей на исходное число пользователей в когорте:

In [ ]:
cohorts['retention'] = cohorts['customer_id'] / cohorts['cohort_users']

**Построим сводную таблицу и создадим тепловую карту**

In [ ]:
retention_pivot = cohorts.pivot_table(index='first_activity_date', columns='cohort_lifetime',
                                                            values='retention', aggfunc='sum').round(3)

In [ ]:
retention_pivot.fillna('')

In [ ]:
sns.set(style='white')
plt.figure(figsize=(16, 10))
plt.title('Retention Rate по когортам', fontsize=15)
ax=sns.heatmap(retention_pivot, annot=True, fmt='.1%', linewidths=1, linecolor='gray', cmap="coolwarm", vmax=0.03)
ax.set_yticklabels(retention_pivot.index.date)
plt.xlabel('cohort_lifetime', fontsize=15)
plt.ylabel('Когорты', fontsize=15);

In [ ]:
mean_retention = retention_pivot[1].mean() * 100
print('Средний Retention Rate на второй месяц «жизни» когорт:', mean_retention.round(2), '%')

**Вывод:**

Первый визит за покупками был 2019-01-27 01:07. Первая активность была 01.10.2018 в кол-ве 181 покупателя.
Среднее количество уникальных покупателей в день - 6. Среднее количество уникальных пользователей в месяц - 198. Среднее количество сессий в день на 2 покупателя.

Retention Rate возвращается от 0.4% до 40.8%.
Средний Retention Rate на второй месяц «жизни» когорт 1.36 % - метрика показывает низкую жизненную ценность.


**Метрики электронной коммерции**

Определим сумму покупок по каждому заказу

In [ ]:
# Получим месяц первой покупки каждого покупателя
first_orders = online_shop.groupby('customer_id').agg({'month':'min'}).reset_index()
first_orders.columns = ['customer_id', 'first_order_month']

In [ ]:
# Посчитаем количество новых покупателей n_buyers за каждый месяц
cohort_sizes = (first_orders.groupby('first_order_month').agg({'customer_id':'nunique'}).reset_index())
cohort_sizes.columns = ['first_order_month', 'n_buyers']
cohort_sizes

Построим когорты. Добавим месяц первой покупки каждого покупателя в таблицу с заказами.

In [ ]:
online_shop = pd.merge(online_shop, first_orders, on='customer_id')

In [ ]:
online_shop['first_order_month'] = online_shop['first_activity_date'].astype('datetime64[M]')

Сгруппируем данные по first_order_month и оценим показатели каждой когорты

In [ ]:
cohort_grouped = online_shop.groupby('first_order_month').agg({'customer_id':'nunique', 'revenue':'sum'}).reset_index()
cohort_grouped.columns = ['first_order_month', 'customer_id', 'cohort_price_sum']
cohort_grouped

Построим сводную таблицу изменения количества покупателей в когорте

In [ ]:
online_shop.pivot_table(index='first_order_month', columns='month', values='customer_id', aggfunc='nunique').fillna('')

В апреле 2019 года было больше всего покупателей в кол-ве 225 ед.(382 481.66 у.е.) Покупателей много, а чек не самый высокий.

In [ ]:
online_shop['first_order_month'] = online_shop['first_order_month'].astype('Datetime64')
online_shop['month'] = online_shop['month'].astype('Datetime64')

In [ ]:
# Сгруппируем данные
cohort_data_grouped = online_shop.groupby(['first_order_month', 'month'])\
                        .agg({'customer_id':'nunique', 'revenue':'sum', 'order_id': 'nunique'}).reset_index()
cohort_data_grouped.head()

In [ ]:
# Найдём `lifetime` когорты
cohort_data_grouped['cohort_lifetime'] = (cohort_data_grouped['month'] - cohort_data_grouped['first_order_month'])

In [ ]:
cohort_data_grouped.head()

In [ ]:
# Преобразуем этот тип в число месяцев, прошедших между датами
cohort_data_grouped['cohort_lifetime'] = cohort_data_grouped['cohort_lifetime'] / np.timedelta64(1, 'M')
# Округлим значения столбца cohort_lifetime, а затем приведём их к целочисленному типу 
cohort_data_grouped['cohort_lifetime'] = (cohort_data_grouped['cohort_lifetime'].round().astype('int'))
cohort_data_grouped.columns = ['first_order_month', 'order_month', 'count_customer_id', 'revenue_sum',
                                'count_order_id','cohort_lifetime']

cohort_data_grouped.head()

**Определим средний чек покупателя**

In [ ]:
cohort_data_grouped['revenue_per_user'] = (cohort_data_grouped['revenue_sum'] / cohort_data_grouped['count_customer_id'])
cohort_data_grouped.round(2).head(10)

In [ ]:
users_orders = cohort_data_grouped.query('cohort_lifetime == 0')[['first_order_month', 'count_customer_id']]
cohort_orders_merge = pd.merge(cohort_data_grouped, users_orders, on='first_order_month')
cohort_orders_merge.drop(['count_customer_id_x'], axis='columns', inplace=True)
cohort_orders_merge.rename(columns={'count_customer_id_y':'count_customer_id'}, inplace=True)
cohort_data_grouped['order_per_user'] = (cohort_data_grouped['count_order_id'] / cohort_data_grouped['count_customer_id']).round(0)
cohort_data_grouped.head()

**Определим среднее количество заказов на пользователя в каждой когорте**

In [ ]:
orders_per_users = cohort_data_grouped.pivot_table(index='first_order_month', 
                             columns='cohort_lifetime', values='order_per_user', aggfunc='sum').round(2)
orders_per_users.fillna('')

In [ ]:
order_month_per_users = cohort_data_grouped.pivot_table(index='first_order_month', columns='order_month', values='order_per_user', aggfunc='sum').round(2)
order_month_per_users.fillna('')

In [ ]:
sns.set(style='white')
plt.figure(figsize=(15, 9))
plt.title('Среднее количество заказов на пользователя', fontsize=15)
ax=sns.heatmap(orders_per_users, annot=True, fmt='.1f', linewidths=1, linecolor='gray', cmap="coolwarm", vmax=45)

#plt.xlabel('')
plt.ylabel('Когорты', fontsize=15)
plt.show()

Когорта 2018-10-01 показала высокие результаты (с ноября по март), что возможно связано с открытием магазина.

**Расчитаем среднюю выручку пользователя**

In [ ]:
cohort_data_grouped.pivot_table(index='first_order_month', 
                                columns='order_month', 
                                values='revenue_per_user', 
                                aggfunc='mean').round(2).fillna('')

**Построим сводную таблицу изменения средней выручки**

In [ ]:
revenue_per_user_pivot = cohort_data_grouped.pivot_table(index='first_order_month', 
                                                           columns='cohort_lifetime',
                                                           values='revenue_per_user',
                                                           aggfunc='mean').round(2)
revenue_per_user_pivot.fillna('')

Cредняя выручка первой когорты 2018-10-01 резко возрастает в октябре и резко снижается к апрелю.

Определим 95-й и 99-й перцентили стоимости заказов

In [ ]:
np.percentile(online_shop['price'], [95, 99])

Всего 5% пользователей совершили покупки от 2399 у.е. 1% пользователей совершили покупки от 5249 у.е. Аномальными значениями будут покупки от 5249 у.е.

**Распределим количество покупок по месяцам**

In [ ]:
purchase_counts = online_shop.groupby('month').agg({'order_id':'nunique'})\
                                     .reset_index().sort_values(by='order_id', ascending=False)
purchase_counts.columns = ['month', 'purchase_quantity']
purchase_counts.head(20)

In [ ]:
trace = go.Bar(
    x = purchase_counts['month'], 
    y = purchase_counts['purchase_quantity'],
    marker_color='#220c74'
)
layout = go.Layout(
    title={
        'text': 'Распределение количества покупок по месяцам',
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
)

fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

Лидируют месяцы с декабрь по апрель, но в январе был небольшой провал. Думаю идёт подготовка к сезону дачному. 

**Определим количество покупателей в каждом месяце**

In [ ]:
customer_count = online_shop.groupby('month')['customer_id'].nunique()\
                                     .reset_index().sort_values(by='customer_id', ascending=False)
customer_count.columns = ['month', 'customer_quantity']
customer_count.head(20)

In [ ]:
trace = go.Bar(
    x = customer_count['month'], 
    y = customer_count['customer_quantity'],
    marker_color='#530c74'
)
layout = go.Layout(
    title={
        'text': 'Распределение количества покупателей по месяцам',
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
)

fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

Магазин чаще всего посещают в декабре - перед новогодними праздниками, в феврале, марте и апреле для подготовки к новому дачному сезону

**Топ-10 самых дорогих товаров**

In [ ]:
online_shop.sort_values(by='price', ascending=False).head(10)

Самые дорогие товары в интернет-магазине - сушилка уличная и сумки-тележки хозяйственные. Самая дорогая сушилка стоит 14 917 у.е.

**Топ-10 покупателей с самой большой выручкой**

In [ ]:
online_shop.groupby('customer_id').agg({'revenue': 'sum'}).sort_values(by='revenue', ascending=False).head(10)

Лидирует один покупатель с суммой 166 988.0 у.е.

**Вывод:** 
Первый визит за покупками был 2019-01-27 01:07. Первая активность была 01.10.2018 в кол-ве 181 покупателя. Среднее количество уникальных покупателей в день - 6. Среднее количество уникальных пользователей в месяц - 198. Среднее количество сессий в день на 2 покупателя. Retention Rate возвращается от 0.4% до 40.3%. Средний Retention Rate на второй месяц «жизни» когорт 1.36 % - метрика показывает низкую жизненную ценность.

В апреле 2019 года было больше всего покупателей в кол-ве 225 ед.(382 481.66 у.е.) Покупателей много, а чек не самый высокий. Когорта 2018-10-01 показала высокие результаты (с ноября по март), что возможно связано с открытием магазина. Cредняя выручка первой когорты 2018-10-01 резко возрастает в октябре и резко снижается к апрелю. Всего 5% пользователей совершили покупки от 2399 у.е. 1% пользователей совершили покупки от 5249 у.е. Аномальными значениями будут покупки от 5249 у.е.

В распределении покупок по месяцам лидируют месяцы с декабрь по апрель, но в январе был небольшой провал. Думаю идёт подготовка к сезону дачному. Магазин чаще всего посещают в декабре - перед новогодними праздниками, в феврале, марте и апреле для подготовки к новому дачному сезону. Самые дорогие товары в интернет-магазине - сушилка уличная и сумки-тележки хозяйственные. Самая дорогая сушилка стоит 14 917 у.е. Лидирует один покупатель с суммой 166 988 у.е.

**4. СЕГМЕНТАЦИЯ ПОКУПАТЕЛЕЙ НА ОСНОВЕ ИХ ПОКУПОК.**

**Определим уникальные наименования товаров**

In [ ]:
unique_items = pd.DataFrame(online_shop['product'].unique(), columns = ['product'])
unique_items.sample(10)

In [ ]:
# Сформируем выборку для словарей
online_shop['item'] = online_shop['product'].apply(lambda x: x.split()[:1]).str.join(', ')
online_shop['item'].to_frame().sample(10)

In [ ]:
# Количество уникальных товаров
online_shop['item'].nunique()

**Разделим товары на категории.**

In [ ]:
garden_tools = ['сушилка', 'гладильный', 'сумка', 'доска', 'горшок', 'сушилк', 'тележка', 'хозяйственный',\
               'искусственный', 'чехол', 'стремян', 'алюминиевый', 'черенок', 'кашпо', 'багажный', \
               'ведро', 'контейнер', 'швабра', 'ведро', 'мытье', 'крючок', 'обув', 'перчатк', 'ящик', 'лоток',\
               ]


In [ ]:
plants = ['цветок', 'рассада', 'среднеспелый', 'томат', 'помидор', 'диам', 'пеларгония', 'розовый',\
          'зелень', 'роза', 'среднеспелый', 'муляж', 'однолетний', 'пластиковый', 'петуния', 'раннеспелый',\
          'растение', 'огурец', 'герань', 'тимьян', 'цикламен', 'калибрахоа', 'флокс', 'тыква', 'клубника',\
          'петрушка', 'фиалка', 'бегония', 'трава', 'флокс', 'перец', 'душица', 'комнатн', 'фуксия', 'капуста',\
          'дыня', 'хлорофитум', 'хризантема', 'зверобой', 'кореопсис', 'цветущее', 'лавр', 'ель', 'антуриум', 'укроп'\
          'осина', 'бадан', 'антинакипин', 'радермахер', 'хризолидокарпус', 'юкка', 'суккулент', \
          'цинерария', 'диффенбахий', 'эхинокактус', 'гипсофил', 'ранункулус', 'эхинацея', 'мускарь',\
          'горох', 'эхинокактус', 'табак', 'петрушка', 'шеффлер', 'многолетний', 'алиссум', 'бальзам', 'настурция',\
          'адиантум', 'кампануть', 'нолина', 'глоксиния', 'иссоп', 'сантолина', 'мелисса', \
          'космея', 'бархатцы', 'монарда', 'гортензия', 'диффенбахий', 'цинерария', 'шалфей', 'валериана', 'бадан',\
          'агератум', 'аргирантерум', 'вероника', 'целозия', 'сальвия', 'фатсия', 'хоста', 'монард', \
          'настольный', 'альбук', 'артемизия', 'калоцефалус', 'каллун', 'капсикум', 'соланум',\
          'бархатцы', 'годеция', 'эшшольция', 'пуансетия', 'кипарисовик', 'фиттоний', 'кодонант',\
          'калла', 'портулак', 'патиссон', 'эхеверие', 'хризантема', 'лаванда', 'кофе', 'овсянница',\
          'суккулент', 'драцена', 'морковь', 'годеция', 'эшшольция', 'комнатный', 'аптение', 'настурция', 'цветущий',\
          'девичий', 'лантан', 'ель', 'фикус', 'лен', 'лапчатка', 'рудбекия', 'цитрофортунелла', 'подсолнечник']

In [ ]:
crockery = ['luminarc', 'тарелк', 'салатник', 'чайник', 'салфетк', 'ложк', 'стакан', 'крышк', 'вилк', 'нож',\
           'сковород', 'кружк', 'термос', 'губка', 'измельчитель', 'миска', 'хлебница', 'сахарн', 'рыбочистка',\
           'соковарка', 'бидон', 'картофелемялка','орехоколка',  'скраб', 'венчик', 'сито',\
           'блюдо','tepмокружка','кухонный','миска','блюдце', \
           'скалка', 'завертка',  'толкушка', 'пружина', 'молодить', 'лопатка', ]

In [ ]:
furniture = ['кухня', 'ванный', 'коврик', 'вешалк', 'потолочный', 'ванна', 'обеденн', 'щетк', 'карниз',\
             'таз', 'штора', 'зубн', 'коробк', 'полка', 'комод', 'лестниц', 'щетка', 'щётка', 'увлажнять',\
             'скоба', 'пресс', 'кольцо', 'фоторамка', 'ключница', 'перекладина', 'модульный', 'фал',   \
             'корыто', 'ковер', 'сидение', 'плед', 'полотенце', 'ваза', 'подушка', 'утюг', 'кисточка', 'шило',\
             'этажерка', 'стойка', 'стеллаж', 'нетканый', 'полк', 'моп', 'тряпкодержатель', 'шпагат', 'шприц', 'крышка',\
             'бензин', 'пробка', 'универсальный', 'сменный', 'кисточка', 'сетка', 'ручка', 'складной', 'просеиватель', 'отжим', \
             'вкладыш', 'стиральный', 'чистить', 'светильник', 'жидкий', 'пена', 'ножеточка', 'линейка', 'салфетница', 'овощечистка', \
             'держатель', 'губка', 'антижир', 'ролик', 'картофелемялка', 'ароматизированный', 'бальзам', 'рассекатель', 'ополаскиватель', \
             'скоба', 'пресс', 'кольцо', 'фоторамка', 'ключница', 'перекладина', 'модульный', 'фал', 'орехоколка', 'увлажнять', \
              'концентрат', 'смолевка', 'фиксатор', 'шар', 'обувница', 'штангенциркуль','совок', 'измерительный',\
             'сотейник', 'ванна', 'прищепок', 'отделитель', 'веревка', 'отбеливатель', 'решетка', 'запасный', 'уголок', 'тряпка', \
             'сахарница', 'половник', 'котел', 'готовый',]

In [ ]:
other = ['']

**Определим количество товаров по категориям**

In [ ]:
def category(row):   
    for i in garden_tools:
        if i in row:
            return "Товары для сада"

    for i in plants:
        if i in row:
            return "Растения"

    for i in crockery:
        if i in row:
            return "Кухонные принадлежности"

    for i in furniture:
        if i in row:
            return "Товары для дома"

    for i in other:
        if i in row:
            return "Другое"

online_shop['category'] = online_shop['product'].str.lower().apply(category)

**Отобразим количество по категориям**

In [ ]:
category_cnt = online_shop['category'].value_counts().to_frame()
category_cnt

In [ ]:
per_group = (online_shop.groupby("category").agg({"customer_id":"count"}).apply(lambda x:100 * x/x.sum())).round(2).reset_index()
per_group.columns = ['Категория', '% по категории']
per_group.sort_values('% по категории', ascending=False)

In [ ]:
labels = per_group['Категория']
values = per_group['% по категории']

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(
    title={
        'text':'Категории товара',
        'y':0.87,
        'x':0.45})
fig.show()

Ожидаемо лидируют категории Растения 50,2% и Товары для сада 26,8%. 

**Определим самые продаваемые товары**

In [ ]:
quantity_top = online_shop.groupby('product')['quantity'].sum().reset_index().sort_values(by='quantity', ascending=False).head(15)
quantity_top

Самый продаваемый товар это муляж яблоко 9 см красное.

**Сгруппируем покупателей по количеству заказов**

In [ ]:
count_order = online_shop.groupby(['customer_id']).agg({'order_id':'nunique'}).reset_index()\
                                          #.sort_values(by='order_id', ascending=False)
count_order.columns=['customer_id', 'order_count']
count_order.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(
    y=count_order["order_count"],
    name="Заказы",
    boxpoints='outliers',
    marker_color='#10d9b4',
    line_color='#220c74'
))

fig.update_layout(xaxis_tickangle=0,
        title={
        'text': 'Диаграмма размаха количества заказов',
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [ ]:
def cnt_quantity (row):
    if row <= 1:
        return '1 заказ'
    if row == 2:
        return '2 заказа'
    if row == 3:
        return '3 заказа'
    if row == 4:
        return '4 заказа' 
    if row == 5:
        return '5 заказов' 
    if row <= 10:
        return 'до 10 заказов'
    if row <= 50:
        return 'до 50 заказов'
    if row <= 100:
        return 'до 100 заказов'
    if row > 100:
        return 'более 100 заказов'
count_order['order_quantity_category'] = count_order['order_count'].apply(cnt_quantity)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=count_order.order_quantity_category, name="count", marker_color='#220c74'))
fig.update_layout(xaxis_title='Категории', yaxis_title='Величина', title_font_size = 20,
        title={
        'text': 'Количество товаров по категориям',
        'y':0.88,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, showlegend=True)
fig.show()

In [ ]:
per_cnt_quantity = (count_order.groupby("order_quantity_category").agg({"customer_id":"count"}).apply(lambda x:100 * x/x.sum())).reset_index()
per_cnt_quantity.columns = ['Категория', '% по категории']
per_cnt_quantity.sort_values('% по категории', ascending=False)

Почти 63.8 % покупателей совершили по 1 заказу, а 2 заказа - 35 %. 

Большинство покупателей совершили по 1 покупке за всё время.

**Выручка и средний чек в разрезе категорий товаров**

In [ ]:
revenue_by_categories = online_shop.groupby('category').agg({'revenue': 'sum', 'customer_id': 'nunique', 'order_id': 'nunique'}) \
                        .sort_values(by='revenue', ascending=False)
revenue_by_categories['avg_check_customer'] = revenue_by_categories.revenue / revenue_by_categories.customer_id
revenue_by_categories['avg_check_order'] = revenue_by_categories.revenue / revenue_by_categories.order_id
revenue_by_categories.columns = ['revenue', 'unique_customers', 'unique_orders', 'avg_check_customer', 'avg_check_order']

revenue_by_categories

Наибольшую выручку принесли товары для сада и товары для дома. 

Не смотря на то что растения покупают чаще всего, средний чек их самый маленький. 

**Изменение выручки по месяцам**

In [ ]:
online_shop.pivot_table(
    index='month', 
    columns='category', 
    values='revenue',  
    aggfunc='sum',  
)

In [ ]:
online_shop.pivot_table(
    index='month', 
    columns='category',  
    values='revenue',  
    aggfunc='sum',  
).plot(figsize=(8, 8), grid=True)

plt.title('Изменение выручки по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Выручка')
plt.legend(loc='best')
plt.show()

Категорию растения покупали больше в апереле-мае, т.к. в эти месяцы начинается дачный сезон. Пик товаров для дома в июне, время отпусков, возможно кто-то делает мини-ремонт. Товары для сада покупали больше всего в декабре, марте и июне. Кухонные принадлежности в ноябре. 

**Изменение количества уникальных покупок по месяцам**

In [ ]:
online_shop.pivot_table(
    index='month',  
    columns='category',  
    values='order_id',  
    aggfunc='nunique')

In [ ]:
online_shop.pivot_table(
    index='month',  
    columns='category',  
    values='order_id',  
    aggfunc='nunique',  
).plot(figsize=(8, 8), grid=True)

plt.title('Изменение количества уникальных покупок по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Количество уникальных покупок')
plt.legend(bbox_to_anchor=(1.02, 1.0), loc='best')
plt.show()

По "уникальным" покупкам в феврале больше всего покупок по категории Товары для сада. Растения покупают в апреле, а ближе к октябрю сильный провал. И также провал по покупкам у всех категорий в январе. В апреле категории Другие, Кухонные принадлежности и товары для дома имели всплекс по покупкам. Также с октября по январь у них есть рост.

**Изменение количества уникальных покупателей по месяцам**

In [ ]:
online_shop.pivot_table(
    index='month',  
    columns='category',  
    values='customer_id',  
    aggfunc='nunique',  
)

In [ ]:
online_shop.pivot_table(
    index='month',  
    columns='category',  
    values='customer_id',  
    aggfunc='nunique',  
).plot(figsize=(10, 10), grid=True)

plt.title('Изменение количества уникальных покупателей по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Количество уникальных покупателей')
plt.legend(bbox_to_anchor=(1.02, 1.0), loc='best')
plt.show()

У всех категория рост в апреле по покупкам, и провалы в январе. 

**Вывод:**

Разделили товар на 6 категорй: Товары для сада, Растения, Кухонные принадлежности, Товары для дома и Другие. Самая большая группа по покупаемым товарам это Растения и Товары для сада. Наибольшую выручку принесли Товары для сада и Товары для дома. Не смотря на то что растения покупают чаще всего, средний чек их самый маленький. 63.8 % покупателей совершили по 1 заказу, а 2 заказа - 35 %. Наибольшую выручку принесли товары для сада и товары для дома. Не смотря на то что растения покупают чаще всего, средний чек их самый маленький.

У всех категория рост в апреле по покупкам, и провалы в январе.

**Создадим рабочую модель**

In [ ]:
# Предварительно проведем необходимые расчеты
price_sum = (online_shop.groupby("order_id").agg({"price":"sum"})).reset_index()
price_sum.columns = ['order_id', 'price_by_order']
price_sum.sort_values('price_by_order', ascending=False).head(10)

In [ ]:
online_shop.columns

In [ ]:
online_shop_merge = online_shop.merge(price_sum, on='order_id')
online_shop_merge.head()

In [ ]:
# Определим количество товара купленного каждым пользователем
quantity_count = (online_shop.groupby("customer_id").agg({"quantity":"sum"})).reset_index()
quantity_count = quantity_count.rename(columns={'quantity':'item_count'}) 
quantity_count.sort_values('item_count', ascending=False).head()

In [ ]:
online_shop_merge = online_shop_merge.merge(quantity_count, on='customer_id')
online_shop_merge.head()

In [ ]:
count_order.head()

In [ ]:
# Добавим информацию о количестве заказов
online_shop_merge = online_shop_merge.merge(count_order, on='customer_id')
online_shop_merge.head()

In [ ]:
# Посчитаем выручку на покупателя
online_shop['revenue'] = online_shop['price'] * online_shop['quantity']
revenue_customer = (online_shop.groupby("customer_id").agg({"revenue":"sum"})).reset_index()
revenue_customer = revenue_customer.rename(columns={'revenue':'revenue_by_customer'}) 
revenue_customer.sort_values('revenue_by_customer', ascending=False).head()

In [ ]:
online_shop_merge = online_shop_merge.merge(revenue_customer, on='customer_id')

In [ ]:
data_category = online_shop_merge.drop(columns = ['date', 'month', 'day', 'first_activity_date', 'order_id',\
                               'first_month', 'first_order_month', 'product', 'item',\
                              'quantity', 'price', 'cohort_lifetime', 'revenue', 'total_price'])

data_category.sample(5)

In [ ]:
level_map = {'Растения':1, 'Товары для сада':1, 'Товары для дома':1, 'Другое':1, 'Кухонные принадлежности':1}
data_category['cat'] = data_category['category'].map(level_map)
data_category.head()

Для определения профилей потребления клиентов, создадим сводную таблицу с категориями товаров

In [ ]:
data_segments = data_category.pivot_table(index=['customer_id','price_by_order', 'item_count', 'revenue_by_customer'],\
                                          columns = 'category', values='cat', aggfunc='mean')\
                                                            .round(0).fillna(0).reset_index()
data_segments[['Товары для дома', 'Кухонные принадлежности', 'Другое', 'Растения', 'Товары для сада']]\
                    = data_segments[['Товары для дома', 'Кухонные принадлежности', 'Другое', 'Растения', 'Товары для сада']].astype(int)
data_segments.head()

In [ ]:
# Оставим только уникальных customer_id
data_segments = data_segments.drop_duplicates(subset = ['customer_id'])

Построим матрицу корреляций

In [ ]:
cm = data_segments.corr()
fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(cm, fmt='.2f', vmin=-1, annot = True, linewidths=1, linecolor='gray', cmap='coolwarm')
plt.title('Матрица корреляций', fontsize = 16)
plt.ylabel('Признаки', fontsize=15)
plt.xlabel('Признаки', fontsize=15)
plt.show()

**Обучим модель кластеризации на основании алгоритма K-Means и спрогнозируем кластеры клиентов**

Для создания рабочей модели необходимо удалить колонку customer_id

In [ ]:
# удалим сильно скореллированные признаки
X = data_segments.drop(['customer_id'], axis=1)
y = data_segments['customer_id']

In [ ]:
# обязательная стандартизация данных перед работой с алгоритмами 
sc = StandardScaler()
X_sc = sc.fit_transform(X)

Построим матрицу расстояний функцией linkage()на стандартизованной матрице признаков, нарисовам дендрограмму

In [ ]:
linked = linkage(X_sc, method = 'ward')

In [ ]:
plt.figure(figsize=(16, 10))  
dendrogram(linked, orientation='top', no_labels=True)
plt.title('Дендрограмма иерархической кластеризации', fontsize=16)
plt.show() 

Обучим модель кластеризации на основании алгоритма K-Means и спрогнозируем кластеры клиентов

Оставила 5 кластеров, т.к у меня 5 категорий товаров и на дендрограмме это тоже видно.

In [ ]:
# За число кластеров примем n=5
n = 5
km = KMeans(n_clusters = n, random_state=0) 
labels = km.fit_predict(X_sc)

In [ ]:
# сохраняем метки кластера в поле нашего датасета
data_segments['cluster_km'] = labels

In [ ]:
silhouette_score(X_sc, labels).round(3)

**Отобразим средние значения признаков кластеров**

In [ ]:
# выводим статистику по средним значениям наших признаков по кластеру
clusters = data_segments.groupby('cluster_km').mean().T.round(2)
clusters

In [ ]:
# Описание количества покупателей
data_segments.groupby("cluster_km").agg({"customer_id":"nunique"}).T

Получили сегментацию по 5 кластерам.

Все 5 кластеров отличаются по цене. По выручке тоже большие различия. В кол-ве заказов ттоже есть отличается от других. Больше всего покупателей у 0 и 1 кластеров. 

**ФОРМУЛИРОВАНИЕ И ПРОВЕРКА СТАТИСТИЧЕСКИХ ГИПОТЕЗ.**

Определим статистическую значимость различий в средней выручке между кластерами

Применим критерий и отформатируем p-value, округлив его до трёх знаков после запятой:

- Н0 - Статистически значимых различий в средней выручке заказа нет
- Н1 - Статистически значимые различия в средней выручке заказа есть

Объявим переменные cluster в соответствии с их price_by_order значениями

In [ ]:
cluster0 = data_segments.query('cluster_km==0')['revenue_by_customer']
cluster1 = data_segments.query('cluster_km==1')['revenue_by_customer']
cluster2 = data_segments.query('cluster_km==2')['revenue_by_customer']
cluster3 = data_segments.query('cluster_km==3')['revenue_by_customer']
cluster4 = data_segments.query('cluster_km==4')['revenue_by_customer']
#cluster5 = data_segments.query('cluster_km==5')['revenue_by_customer']

Проведем тест «U-критерий Манна-Уитни»

In [ ]:
alpha = 0.05 # критический уровень статистической значимости (стандарт для экономических данных-0.05)

clusters = [cluster0, cluster1, cluster2, cluster3, cluster4]
cluster_list = ['cluster0', 'cluster1', 'cluster2', 'cluster3', 'cluster4']

bonferroni_alpha = alpha / 5 

for index_0 in range(len(clusters)):
    for index_1 in range(index_0+1, len(clusters)):
        p_value = st.mannwhitneyu(clusters[index_0], clusters[index_1], alternative='two-sided')[1]

        print('Кластеры', cluster_list[index_0], cluster_list[index_1])
        print('p_value: {:.3f}'.format(p_value))
        print('Средняя выручка группы A: {:.0f}'.format(clusters[index_0].mean()))
        print('Средняя выручка группы B: {:.0f}'.format(clusters[index_1].mean()))
        
        if p_value < bonferroni_alpha:
            print('\033[31m' + 'Отвергаем нулевую гипотезу: разница статистически значима\n'+ '\033[0m')
        else:
            print('\033[32m' + 'Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя\n'+ '\033[0m')

В результате проведения теста Манна-Уитни получили, что в некоторых случаях мы не отвергаем нулевую гипотезу. Что означает: значимые различия между сравниваемыми выборками есть.

Кластерам 0 и 1, 0 и 3, 1 и 3, 2 и 3, 3 и 4 могут соответствовать разовые покупатели.

Кластерам 0 и 2, 0 и 4, 1 и 2, 1 и 4, 2 и 4  могут соответствовать условно постоянные покупатели.

**ОБЩИЙ ВЫВОД:**

В таблице содержатся данные о заказах покупателей в интернет-магазине за период с 1 октября 2018 г. до 31 января 2020 г.

Среднее значение товаров - 2.33. Стоимость товара составляет от 9 у.е. до 14 917. Медиана - 135 у.е., среднее значение - 474.01.

В среднем выручка составляет 68 760 у.е. за товарную позицию (не заказа). Самый крупный заказ составил 112 789 у.е.


Общее количество уникальных покупателей - 2412 ед. А количество уникальных товаров - 2331 шт. Количество уникальных заказов - 3491 шт. На графике мы видим, что в одной покупке встречается чаще 1 единица товара.

Наибольшее количество заказов были сделаны покупателями в в декабре 2018 г., феврале 2019 г., в марте и апреле 2019 г. новогодние праздники и весна(начало дачного сезона). Большой провал в январе и июне. И снова подъем с ноября.

Средняя выручка или чек с заказа составляет 1121.37 у.е., медиана - 580 у.е. Минимальная выручка с покупки - 9 у.е., максимальная - 49 432 у.е.
5% заказов принесли выручку не менее 3768 у.е. 1% заказов - не менее 7506.1 у.е.

Один покупатель принес за весь период в среднем 1623.02 у.е., медиана - 837 у.е. Минимальная сумма - 15 у.е., максимальная - 166988 у.е.
5% пользователей принесли выручку не менее 4885.7 у.е. 1% пользователей - не менее 9867.37 у.е.

Выручка в ноябре 2018 составила 359 348 у.е - самый высокий показатель Самый большой средний чек покупателя в ноябре 2018 и составил 2018.80 у.е. хотя кол-во покупок не самое большое.


Первый визит за покупками был 2019-01-27 01:07. Первая активность была 01.10.2018 в кол-ве 181 покупателя. Среднее количество уникальных покупателей в день - 6. Среднее количество уникальных пользователей в месяц - 198. Среднее количество сессий в день на 2 покупателя. Retention Rate возвращается от 0.4% до 40.3%. Средний Retention Rate на второй месяц «жизни» когорт 1.36 % - метрика показывает низкую жизненную ценность.

В апреле 2019 года было больше всего покупателей в кол-ве 225 ед.(382 481.66 у.е.) Покупателей много, а чек не самый высокий. Когорта 2018-10-01 показала высокие результаты (с ноября по март), что возможно связано с открытием магазина. Cредняя выручка первой когорты 2018-10-01 резко возрастает в октябре и резко снижается к апрелю. Всего 5% пользователей совершили покупки от 2399 у.е. 1% пользователей совершили покупки от 5249 у.е. Аномальными значениями будут покупки от 5249 у.е.

В распределении покупок по месяцам лидируют месяцы с декабрь по апрель, но в январе был небольшой провал. Думаю идёт подготовка к сезону дачному. Магазин чаще всего посещают в декабре - перед новогодними праздниками, в феврале, марте и апреле для подготовки к новому дачному сезону. Самые дорогие товары в интернет-магазине - сушилка уличная и сумки-тележки хозяйственные. Самая дорогая сушилка стоит 14 917 у.е. Лидирует один покупатель с суммой 166 988 у.е.

Получили сегментацию по 5 кластерам.
Все 5 кластеров отличаются по цене. По выручке тоже большие различия. В кол-ве заказов ттоже есть отличается от других. Больше всего покупателей у 0 и 1 кластеров.

В результате проведения теста Манна-Уитни получили, что в некоторых случаях мы не отвергаем нулевую гипотезу. Что означает: значимые различия между сравниваемыми выборками есть.
Кластерам 0 и 1, 0 и 3, 1 и 3, 2 и 3, 3 и 4 могут соответствовать разовые покупатели.
Кластерам 0 и 2, 0 и 4, 1 и 2, 1 и 4, 2 и 4 могут соответствовать условно постоянные покупатели.

**Рекомендации по кластерам:**
- Кластер0, категория Другие. В ходе исследования выяснилось, что категория не доисследована(нужно было разделить Другие на более маленькие категории товаров, чтоб сделать вывод по ним). Хотя покупателей достаточно (704 ед.) и продаются хорошо(1274.22 у.е). 


- Кластер1, категория Растения. Покупают много по количеству, но средняя выручкав чеке не такая высокая (1622.17 у.е). Эту категорию необходимо предлагать сопутствующие товары например к категориям Товаров для сада и дома. Кол-во покупателей 814 ед.


- Кластер2, категория Товары для дома. Одна из лидирующих категорий, хорошая средняя выручка в чеке(1381.66 у.е). Продается круглогодично. Можно придумать программы лояльности и устраивать акции. А также увеличить ассортимент. Кол-во покупателей 210 ед.


- Кластер3, категория Товары для сада. Весной усиливать работу над привлечением покупателей в категории. Покупателям можно предлогать товар основываясь на предыдущих покупках. Прекрасная средняя выручка в чеке 2140.24 у.е. Кол-во покупателей 313 ед.


- Кластер4, категория Кухонные принадлежности. Крупногоборитные товары покупаются меньше, а "мелкие" товары берутся в большом количестве, но выручки в чеке много не приносят(1987.05 у.е.). Кол-во покупателей371. Судя можно еще рекомендовать сопуствующие товары, т.к. чаще покупатели кух.предметов домохозяйки и можно рекомендовать категорию растения. 